In [33]:
from pandas import to_datetime #good flexible date interpereter
import requests
import sys
import os
import csv
from time import sleep

In [81]:
input_filepath = "fail_tests.csv"
input_filepath = "test-data.csv"
output_filepath = "out.txt"

In [84]:
# given dict with ["Account ID", "Created On", "First Name"] keys,
# convert Account ID to integer and Created On to formatted datestring
def verify_and_clean_input(input_dict):
    output_dict = {}
    try:
        output_dict["Account ID"] = int(input_dict["Account ID"])
        if output_dict["Account ID"] < 0:
            return False, {}, "not a valid account id"
    except ValueError:
        return False, {}, "not a valid account id"
    
    try:
        output_dict["Created On"] = to_datetime(input_dict["Created On"], errors='coerce').strftime("%x")
    except ValueError:
        output_dict["Created On"] = ""
    
    output_dict["First Name"] = input_dict["First Name"]
        
    return True, output_dict, ""

verify_and_clean_input({"Account ID": 314159,"Account Name": "superman","First Name": "Ka-el","Created On": "1/12/13"})
#(True, {'Account ID': 314159, 'Created On': '01/12/13', 'First Name': 'Ka-el'},'')

(True,
 {'Account ID': 314159, 'Created On': '01/12/13', 'First Name': 'Ka-el'},
 '')

In [87]:
# combines input_dict with status and created_on from a query response dict
# maps status -> "Status", and created_on -> "Status Set On"
# returns filled out dict and success/failure message
def extend(input_dict, response_dict):
    output_dict = input_dict
    output_dict["Status"] = ""
    output_dict["Status Set On"] = ""
    message = response_dict["message"]
    account_id = input_dict["Account ID"]

    if response_dict["valid"]: 
        # set Status Set On to response's created_on value for account_id
        # could introduce check for status in set ("good", "bad", "")
        if "status" in response_dict:
            output_dict["Status"] = response_dict["status"]
            message = f"Processed Account ID: {account_id} - Status: {response_dict['status']} "
        else:
            message = f"No status data included for Account ID - {account_id} "
            
        # only try and set status date if status successfully set
        if "created_on" in response_dict and "status" in response_dict:
            try:
                # set Status Set On to response's created_on value for account_id
                # here choosing to write "" in event of bad date
                output_dict["Status Set On"] = \
                    to_datetime(response_dict["created_on"], errors='raise')\
                    .strftime("%x")
            except ValueError:
                # has status and created on but created on not valid
                message = f"Invalid date included for Account ID - {account_id}"
        else:
            if "status" not in response_dict:
                message = f"No created_on date included for Account ID - {account_id}"
                        
    return output_dict, message
print(extend({'Account ID': 314159, 'Created On': '01/12/13', 'First Name': 'Ka-el'}, {'account_id': 314159, 'status': 'good', 'created_on': '2012-01-12', 'message': '', 'valid': True}))
#({'Account ID': 314159, 'Created On': '01/12/13', 'First Name': 'Ka-el', 'Status': 'good', 'Status Set On': '01/12/12'}, 'Processed Account ID: 314159 - Status: good ')
print(extend({"Account ID": 271},{'message': "", "valid": True, 'status': 'good', 'created_on': '2011-03-22'}))
#({'Account ID': 271, 'Status': 'good', 'Status Set On': '03/22/11'}, 'Processed Account ID: 271 - Status: good ')

({'Account ID': 314159, 'Created On': '01/12/13', 'First Name': 'Ka-el', 'Status': 'good', 'Status Set On': '01/12/12'}, 'Processed Account ID: 314159 - Status: good ')
({'Account ID': 271, 'Status': 'good', 'Status Set On': '03/22/11'}, 'Processed Account ID: 271 - Status: good ')


In [88]:
# query accounts/account_id and determine if non-error json is returned
# returns validity of response and the response
def query(account_id):
    data_is_valid = True
    query_success = False
    returned_dict = {}
    message = ""
    attempt_num = 0
    
    while (attempt_num < 3) and not query_success:
        sleep(3**attempt_num-1) #backoff [0,2,8] seconds
        attempt_num += 1
        query_string = f"http://interview.wpengine.io/v1/accounts/{account_id}"
        request = requests.get(query_string)

        if request.status_code == 200:
            returned_dict = request.json()
            data_is_valid = True
            query_success = True
        else:
            data_is_valid = False
        
        if request.status_code >= 400:
            try: # check parseable response
                returned_dict = request.json()
                message = f"Query - {query_string} - {returned_dict['detail']}"
                query_success = True # but data not found
            except ValueError:
                message = f"Response not json parseable for query {query_string}"
                query_success = False # requery
    
    returned_dict["message"] = message
    returned_dict["valid"] = data_is_valid
    return returned_dict
print(query(271))
#{'account_id': 271, 'status': 'good', 'created_on': '2011-03-22', 'message': '', 'valid': True}
print(query(314159))
#{'account_id': 314159, 'status': 'good', 'created_on': '2012-01-12', 'message': '', 'valid': True}
print(query(21))
#{'detail': 'Not found.', 'message': 'Account ID - 21 - Not found.', 'valid': False}

{'account_id': 271, 'status': 'good', 'created_on': '2011-03-22', 'message': '', 'valid': True}
{'account_id': 314159, 'status': 'good', 'created_on': '2012-01-12', 'message': '', 'valid': True}
{'detail': 'Not found.', 'message': 'Account ID - 21 - Not found.', 'valid': False}


In [75]:
def main(args):
    
    if len(args) != 3:
        return print("ERROR - invoke program as: wpe_merge <input_file.csv> <output_file.csv>")
    input_filepath = args[1]
    output_filepath = args[2]
    
    '''if not os.path.exists(input_filepath):
        return print(f"ERROR - input filepath {input_filepath} does not exist")
    
    path_to_output = output_filepath.rpartition("/")[0]
    if path_to_output != "":
        os.makedirs(path_to_output, exist_ok=True)
    
    if input_filepath[-4:] != ".csv" or output_filepath[-4:] != ".csv": 
        return print("ERROR - filepaths must be valid csv files")'''
    
    with open(input_filepath, newline="") as input_csv:
        csvreader = csv.reader(input_csv, delimiter=",",)

        needed_input_columns = ["Account ID","First Name", "Created On"]
        needed_output_columns = ["Account ID","First Name", "Created On", "Status", "Status Set On"]
        headers = next(csvreader) #grab first row as headers
        if not set(needed_input_columns).issubset(headers):
            print('ERROR - input csv must contain columns ["Account ID","First Name", "Created On"] as headers')

        with open(output_filepath, mode = "w", newline = "") as output_csv:
            csvwriter = csv.DictWriter(output_csv, fieldnames = needed_output_columns)
            csvwriter.writeheader()

            index_of = {}
            for index,header in enumerate(headers):
                index_of[header] = index
            write_dict = {}

            #Loop through inputfile
            for row in csvreader:
                still_valid = True
                if len(row) != len(headers):
                    message = "ERROR - csv row has incomplete data"
                    still_valid = False
                if still_valid:
                    # extract data from row, columns can be in any order
                    for column in needed_input_columns:
                        write_dict[column] = row[index_of[column]]
                    still_valid, write_dict, message = verify_and_clean_input(write_dict)
                if still_valid:
                    write_dict, message = extend(write_dict, query(write_dict["Account ID"]))
                    #only write to csv if all input data valid, query data nulled out if invalid
                    csvwriter.writerow(write_dict) 
                print(message)

            output_csv.close()
        input_csv.close()
        

In [82]:
main(["",input_filepath, output_filepath])

http://interview.wpengine.io/v1/accounts/314159
Processed Account ID: 314159 - Status: good 
http://interview.wpengine.io/v1/accounts/271
Processed Account ID: 271 - Status: good 
http://interview.wpengine.io/v1/accounts/8675309
Processed Account ID: 8675309 - Status: closed 
http://interview.wpengine.io/v1/accounts/99999
Processed Account ID: 99999 - Status: fraud 


In [ ]:
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')
